# Compare methods on CASP dataset

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
pd.options.display.float_format = '{:.3f}'.format
plt.rcParams["figure.dpi"] = 300
sns.set(style='darkgrid')
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
def eval(df: pd.DataFrame, base_method: str, comparison_method_list: list, label: str = 'GDT_TS', target_column_name: str = 'target', discard_low_model: bool = True, discard_thre=40):
    if discard_low_model:
        df = df.groupby(target_column_name).filter(lambda x: x[label].max() > discard_thre)
    group = df.groupby(target_column_name)
    pearson = group.corr()[label]
    spearman = group.corr('spearman')[label]
    pro_pearson = pearson.mean(level=1)[base_method]
    pro_spearman = spearman.mean(level=1)[base_method]
    loss = group.apply(lambda x: x[label].max()-x[label][x[base_method].idxmax()])
    pro_loss = loss.mean()
    zscore = group.apply(lambda x: stats.zscore(x[label])[x.index.get_loc(x[base_method].idxmax())])
    pro_zscore = zscore.mean()
    pearson_list = [pro_pearson]
    spearman_list = [pro_spearman]
    loss_list = [pro_loss]
    zscore_list = [pro_zscore]
    p_pearson_list = [None]
    p_spearman_list = [None]
    p_loss_list = [None]
    p_zscore_list = [None]
    for method in comparison_method_list:
        pearson_list.append(pearson.mean(level=1)[method])
        _, p_pearson = stats.wilcoxon(list(pearson[:, base_method]), list(pearson[:,method]))
        p_pearson_list.append(p_pearson)
        spearman_list.append(spearman.mean(level=1)[method])
        _, p_spearman = stats.wilcoxon(list(spearman[:, base_method]), list(spearman[:,method]))
        p_spearman_list.append(p_spearman)
        com_loss = group.apply(lambda x: x[label].max() - x[label][x[method].idxmax()])
        loss_list.append(com_loss.mean())
        _, p_loss = stats.wilcoxon(list(loss), list(com_loss))
        p_loss_list.append(p_loss)
        com_zscore = group.apply(lambda x: stats.zscore(x[label])[x.index.get_loc(x[method].idxmax())])
        zscore_list.append(com_zscore.mean())
        _, p_zscore = stats.wilcoxon(list(zscore), list(com_zscore))
        p_zscore_list.append(p_zscore)
        method = [base_method] + list(comparison_method_list)
    
    result_df = pd.DataFrame({'pearson': pearson_list, 'spearman': spearman_list, 'loss': loss_list, 'zscore': zscore_list, 'pearson_p': p_pearson_list, 'spearman_p': p_spearman_list, 'loss_p': p_loss_list, 'zscore_p': p_zscore_list}, index=method)
    return result_df

In [3]:
casp13_df = pd.read_csv('casp13.csv', index_col=0)
casp12_df = pd.read_csv('casp12.csv', index_col=0)
casp11_df = pd.read_csv('casp11.csv', index_col=0)

In [4]:
# casp13_result_df = eval(casp13_df)
casp13_result_df = eval(casp13_df, base_method='Proposed', comparison_method_list=casp13_df.columns[3: ])
casp13_result_df['dataset'] = 'CASP13'
casp13_result_df

,pearson,spearman,loss,zscore,pearson_p,spearman_p,loss_p,zscore_p,dataset
Proposed,0.797,0.757,5.708,1.264,NaN,NaN,NaN,NaN,CASP13
ProQ3D,0.686,0.638,9.482,0.990,0.000,0.000,0.022,0.023,CASP13
ProQ2D,0.708,0.665,8.766,1.000,0.000,0.000,0.012,0.009,CASP13
ProQ3,0.632,0.599,7.891,1.051,0.000,0.000,0.006,0.008,CASP13
ProQ2,0.671,0.634,10.700,0.803,0.000,0.000,0.000,0.000,CASP13
VoroMQA,0.676,0.624,12.105,0.786,0.000,0.000,0.002,0.001,CASP13
SBROD,0.674,0.637,10.014,0.930,0.000,0.000,0.000,0.001,CASP13
Sato-3DCNN,0.748,0.703,6.527,1.167,0.000,0.000,0.444,0.393,CASP13


In [5]:
casp12_result_df = eval(casp12_df, base_method='Proposed', comparison_method_list=casp12_df.columns[3: ])
casp12_result_df['dataset'] = 'CASP12'
casp12_result_df

,pearson,spearman,loss,zscore,pearson_p,spearman_p,loss_p,zscore_p,dataset
Proposed,0.856,0.782,4.319,1.240,NaN,NaN,NaN,NaN,CASP12
ProQ3D,0.750,0.672,7.989,0.922,0.000,0.000,0.005,0.007,CASP12
ProQ3,0.686,0.618,5.292,1.137,0.000,0.000,0.410,0.418,CASP12
ProQ2,0.694,0.616,5.280,1.130,0.000,0.000,0.288,0.281,CASP12
ProQ2D,0.733,0.657,6.396,1.017,0.000,0.000,0.012,0.007,CASP12
VoroMQA,0.671,0.592,7.649,0.963,0.000,0.000,0.043,0.043,CASP12
SBROD,0.682,0.612,7.063,0.967,0.000,0.000,0.035,0.042,CASP12
Sato-3DCNN,0.746,0.675,5.530,1.139,0.000,0.000,0.489,0.499,CASP12


In [6]:
casp11_result_df = eval(casp11_df, base_method='Proposed', comparison_method_list=casp11_df.columns[3: ])
casp11_result_df['dataset'] = 'CASP11'
casp11_result_df

,pearson,spearman,loss,zscore,pearson_p,spearman_p,loss_p,zscore_p,dataset
Proposed,0.724,0.688,3.116,1.289,NaN,NaN,NaN,NaN,CASP11
ProQ3D,0.526,0.496,6.133,0.820,0.000,0.000,0.003,0.008,CASP11
ProQ3,0.476,0.458,5.259,1.080,0.000,0.000,0.034,0.084,CASP11
ProQ2,0.460,0.441,6.769,0.883,0.000,0.000,0.002,0.004,CASP11
ProQ2D,0.481,0.457,6.298,0.859,0.000,0.000,0.001,0.001,CASP11
VoroMQA,0.469,0.466,6.820,0.806,0.000,0.000,0.002,0.003,CASP11
SBROD,0.449,0.434,6.030,0.873,0.000,0.000,0.009,0.007,CASP11
Sato-3DCNN,0.600,0.583,3.662,1.231,0.000,0.000,0.196,0.269,CASP11


In [7]:
casp_result_df = pd.concat([casp13_result_df, casp12_result_df, casp11_result_df])
casp_result_df.to_csv('casp_result.csv')

# compare validation result

In [8]:
val_pro_df = pd.read_csv('val/each_feature/proposed_2.csv', index_col=0).rename({'global_score': 'atom_pssm_local'}, axis=1)
val_atom_only_df = pd.read_csv('val/each_feature/atom_only_4.csv', index_col=0).rename({'global_score': 'atom'}, axis=1)
val_add_pssm_df = pd.read_csv('val/each_feature/add_pssm_3.csv', index_col=0).rename({'global_score': 'atom_pssm'}, axis=1)
val_without_pssm_df = pd.read_csv('val/each_feature/without_pssm_5.csv', index_col=0).rename({'global_score': 'atom_local'}, axis=1)
val_without_atom_df = pd.read_csv('val/each_feature/without_atom_5.csv', index_col=0).rename({'global_score': 'pssm_local'}, axis=1)
val_df = pd.concat([val_pro_df, val_atom_only_df, val_add_pssm_df, val_without_pssm_df, val_without_atom_df], axis=1)
val_df = val_df.loc[:, ~val_df.columns.duplicated(keep='last')]
val_df = val_df.rename({'global_label': 'GDT_TS', 'target_name': 'target'}, axis=1)
val_df['GDT_TS'] *= 100
val_df

,atom_pssm_local,atom,atom_pssm,atom_local,pssm_local,GDT_TS,target,model_path
0,0.201,0.177,0.163,0.262,0.237,32.140,casp7/T0347,casp7/T0347/ROBETTA_TS3.npz
1,0.003,0.052,0.019,0.047,0.006,5.190,casp9/T0547,casp9/T0547/YASARA_TS1.npz
2,0.198,0.161,0.104,0.273,0.222,21.860,casp10/T0674,casp10/T0674/SAM-T06-server_TS1.npz
3,0.571,0.384,0.273,0.592,0.520,33.780,casp10/T0751,casp10/T0751/PconsD_TS4.npz
4,0.866,0.829,0.894,0.833,0.882,71.770,casp7/T0359,casp7/T0359/3D-JIGSAW_RECOM_TS5.npz
...,...,...,...,...,...,...,...,...
23089,0.917,0.754,0.829,0.900,0.890,83.490,casp9/T0538,casp9/T0538/Pcons_TS3.npz
23090,0.023,0.097,0.057,0.088,0.030,8.160,casp10/T0715,casp10/T0715/RBO-MBS_TS1.npz
23091,0.782,0.644,0.662,0.733,0.766,71.230,casp9/T0538,casp9/T0538/gws_TS1.npz
23092,0.020,0.024,0.012,0.035,0.022,11.800,casp10/T0741,casp10/T0741/BhageerathH_TS4.npz


In [9]:
val_df.to_csv('val/val_comparison_for_each_feature.csv')

In [10]:
val_df = pd.read_csv('val/val_comparison_for_each_feature.csv', index_col=0)

In [11]:
def val_eval(df, label_name='GDT_TS'):
    df = df.groupby('target').filter(lambda x: x[label_name].max() > 0.4)
    group = df.groupby('target')
    columns = df.columns[: 5]
    pearson = group.corr()[label_name].mean(level=1)[columns]
    spearman = group.corr(method='spearman')[label_name].mean(level=1)[columns]
    loss = list(map(lambda column: group.apply(lambda x: x[label_name].max() - x[label_name][x[column].idxmax()]).mean(), columns))
    zscore = list(map(lambda column: group.apply(lambda x: stats.zscore(x[label_name])[x.index.get_loc(x[column].idxmax())]).mean(), columns))
    performance_df = pd.DataFrame({'Pearson': pearson, 'Spearman': spearman, 'loss': loss, 'zscore': zscore}, index=columns)
    print(len(group))
    return performance_df

In [12]:
val_eval(val_df)

85


,Pearson,Spearman,loss,zscore
atom_pssm_local,0.865,0.751,2.519,4.866
atom,0.757,0.645,8.518,4.244
atom_pssm,0.834,0.729,9.860,4.239
atom_local,0.847,0.724,3.883,4.742
pssm_local,0.858,0.742,4.818,4.666


In [14]:
val_result_df = eval(val_df, base_method='atom_pssm_local', comparison_method_list=val_df.columns[1: 5])
val_result_df

,pearson,spearman,loss,zscore,pearson_p,spearman_p,loss_p,zscore_p
atom_pssm_local,0.865,0.751,2.519,4.866,NaN,NaN,NaN,NaN
atom,0.757,0.645,8.518,4.244,0.000,0.000,0.002,0.002
atom_pssm,0.834,0.729,9.860,4.239,0.000,0.001,0.000,0.000
atom_local,0.847,0.724,3.883,4.742,0.000,0.000,0.398,0.398
pssm_local,0.858,0.742,4.818,4.666,0.304,0.126,0.005,0.005


In [15]:
val_result_df.to_csv('val/val_result.csv')